In [1]:
import sys
sys.path.append('./models')
from models.BBSNet_model import BBSNetTransformerAttention
from models.BBSNet_model import BBSNetChannelSpatialAttention
from BBSNet_train import setup_gpu, get_model, get_data, main
from BBSNet_test import test
from utils import evaluate, draw_curve

%load_ext autoreload
%autoreload 2

/home/mahdi/Documents/pytorch/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Train

In [2]:
setup_gpu()
model, optimizer, CE = get_model()
train_loader, test_loader, total_step = get_data()
main(optimizer, model, train_loader, test_loader, CE, total_step)

USE GPU 0


/home/mahdi/Documents/pytorch/lib/python3.12/site-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


load data...
Start train...
2025-08-28 01:12:29.286139 Epoch [001/002], Step [0001/1092], Loss1: 0.7062 Loss2: 0.6847
2025-08-28 01:12:50.702722 Epoch [001/002], Step [0100/1092], Loss1: 0.4431 Loss2: 0.3250
2025-08-28 01:13:12.412272 Epoch [001/002], Step [0200/1092], Loss1: 0.1512 Loss2: 0.2336
2025-08-28 01:13:34.178715 Epoch [001/002], Step [0300/1092], Loss1: 0.3123 Loss2: 0.4368
2025-08-28 01:13:56.045793 Epoch [001/002], Step [0400/1092], Loss1: 0.9203 Loss2: 0.3998
2025-08-28 01:14:17.860297 Epoch [001/002], Step [0500/1092], Loss1: 0.2675 Loss2: 0.1463
2025-08-28 01:14:39.661542 Epoch [001/002], Step [0600/1092], Loss1: 0.2903 Loss2: 0.2493
2025-08-28 01:15:01.494023 Epoch [001/002], Step [0700/1092], Loss1: 0.3310 Loss2: 0.2097
2025-08-28 01:15:23.346722 Epoch [001/002], Step [0800/1092], Loss1: 0.2508 Loss2: 0.1959
2025-08-28 01:15:45.127962 Epoch [001/002], Step [0900/1092], Loss1: 0.3667 Loss2: 0.3072
2025-08-28 01:16:06.937059 Epoch [001/002], Step [1000/1092], Loss1: 0.4

# Use Trained Model

I save the model using this code:
torch.save(model.state_dict(), save_path +
                           'BBSNet_epoch_best.pth')

Now, after a long training period, I want to use it on my test data. So, how can I load this model?

In [3]:
from models.BBSNet_model import BBSNetSwin

model = BBSNetSwin()

# assume your model is called `model`
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")
print(f"Non-trainable parameters: {total_params - trainable_params}")


ImportError: cannot import name 'BBSNetSwin' from 'models.BBSNet_model' (/home/mahdi/Documents/bach_prj/repo/BBS-Net/models/BBSNet_model.py)

In [ ]:
# test(method='bbsnet') 